In [1]:
#%pip install requests
#%pip install lxml
import lxml
import requests
import pandas as pd
from bs4 import BeautifulSoup
#Gets the table from one year ofdata
def get_ratings(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(r.content, "html.parser")
    table = soup.find("table", {"id": "ratings-table"})
    
    df = pd.read_html(str(table))[0]
    col_names = ["Rk", "Team", "Conf", "W-L", "AdjEM", "AdjO", "AdjO Rank", "AdjD", "AdjD Rank", "AdjT", "AdjT Rank", "Luck", "Luck Rank", "AdjEM", "AdjEM Rank", "OppO", "OppO Rank", "OppD", "OppD Rank", "AdjEM", "AdjEM Rank"]
    df.columns = col_names
    
    #Remove teams without seeding
    df = df[df['Team'].str[-1].str.isdigit() == True].reset_index(drop=True)
    
    df[['Team','Seed']] = df["Team"].str.rsplit(" ", n=1, expand=True)
    
    return df

def get_multiple_years(start, end):
    dfs = []
    for year in range(start, end+1):
        #Exclude 2020 data since there was no tournament this year
        if year != 2020:
            df = get_ratings(f"https://kenpom.com/index.php?y={year}")
            df["Year"] = year
            dfs.append(df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [2]:
pomeroy_data = get_multiple_years(2002, 2022)

In [3]:
pomeroy_data.to_csv("pomeroy_data.csv")